In [1]:
import pandas as pd
import os
import string
from collections import Counter
from time import time

In [2]:
from textblob import TextBlob
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

In [3]:
import gensim
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [4]:
import spacy
nlp = spacy.load("fr_core_news_sm")

In [5]:
dates_GDN = ["31-01-19","06-02-19","17-02-19","02-03-19","08-03-19","21-03-19"]
sujets_GDN = {"ecologie": "LA_TRANSITION_ECOLOGIQUE",
             "democratie": "DEMOCRATIE_ET_CITOYENNETE",
             "fisc": "LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES",
             "etat": "ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS"}
sujets_VD = os.listdir("data/VD")

In [6]:
flatten = lambda l: [item for sublist in l for item in sublist]

### Fichier utilisé actuellement pour les tests:

`2019-03-04_justice-police-armee_consultation-3.csv`

In [7]:
df = pd.read_csv(f"data/VD/2019-03-04_justice-police-armee_consultation-3.csv")

/home/benjamin/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,11,12,13,34,35,36,42,43,46,47,50,51,54,55,56,57,59,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df

,type,contributions_id,contributions_author_id,contributions_section_title,contributions_title,contributions_bodyText,contributions_createdAt,contributions_updatedAt,contributions_url,contributions_published,...,contributions_sources_body,contributions_sources_createdAt,contributions_sources_updatedAt,contributions_sources_published,contributions_sources_votesCount,contribution_versions_id,contribution_versions_title,contribution_versions_bodyText,contribution_versions_createdAt,contribution_versions_updatedAt
0,opinion,2a6dce90-3c3a-11e9-988a-fa163e5b3bfd,VXNlcjo3MzIwNTEwZi0zYzBlLTExZTktOTg4YS1mYTE2M2...,"Justice, Police, Armée",Fin de la dissuasion nucléaire,Mettre plutôt nos investissements dans les out...,2019-03-01 16:53:32,NaN,https://le-vrai-debat.fr/consultations/justice...,Yes,...,NaN,2019-03-01 16:53:32,NaN,Yes,1.0,2a6dce90-3c3a-11e9-988a-fa163e5b3bfd,Fin de la dissuasion nucléaire,Mettre plutôt nos investissements dans les out...,2019-03-01 16:53:32,NaN
1,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,opinion,d2a3afbb-3c39-11e9-988a-fa163e5b3bfd,VXNlcjo3MzIwNTEwZi0zYzBlLTExZTktOTg4YS1mYTE2M2...,"Justice, Police, Armée",Retour des forces militaires françaises engagé...,Mettre fin à une politique militaire d'ingéren...,2019-03-01 16:51:05,NaN,https://le-vrai-debat.fr/consultations/justice...,Yes,...,NaN,2019-03-01 16:51:05,NaN,Yes,1.0,d2a3afbb-3c39-11e9-988a-fa163e5b3bfd,Retour des forces militaires françaises engagé...,Mettre fin à une politique militaire d'ingéren...,2019-03-01 16:51:05,NaN
3,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,opinion,bf49a981-3c1a-11e9-988a-fa163e5b3bfd,VXNlcjpkNzcwOTFjYi0yOWZlLTExZTktOTg4YS1mYTE2M2...,"Justice, Police, Armée",Gestion citoyenne radars fixes,Création de groupes de citoyens non profession...,2019-03-01 13:08:38,NaN,https://le-vrai-debat.fr/consultations/justice...,Yes,...,NaN,2019-03-01 13:08:38,NaN,Yes,1.0,bf49a981-3c1a-11e9-988a-fa163e5b3bfd,Gestion citoyenne radars fixes,Création de groupes de citoyens non profession...,2019-03-01 13:08:38,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33002,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33003,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33004,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33005,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
body = df[["contributions_bodyText"]].dropna()
body.reset_index(drop=True, inplace=True)

In [10]:
body.shape[0]

1400

-> 1400 Propositions au total

In [11]:
#Les propositions tokenisées
start = time()
body["tokens"] = body["contributions_bodyText"].apply(lambda doc: [mot for mot in nlp(doc) if not (mot.is_stop or mot.is_punct or len(mot) <3)])
print(f"Time: {time()-start:.2f}s")

Time: 44.44s


In [12]:
#Lemme de chaque token
body["lemmas"] = body["tokens"].apply(lambda tokens: [token.lemma_ for token in tokens])

In [13]:
#Le part-of-speech tag pour chaque token, avec son lemme
body["tags"] = body["tokens"].apply(lambda tokens: [(token.pos_, token.lemma_) for token in tokens if token.pos_ not in ("PUNCT", "SPACE")])

In [14]:
#Nombre de tokens par proposition
body["propLen"] = body["tokens"].apply(lambda l: len(l))

In [15]:
#Nombre de phrases par proposition
body["nbPhrases"] = body["contributions_bodyText"].apply(lambda doc: len(TextBlob(doc).sentences))

In [16]:
#unigram
body["unigram"] = body["lemmas"].apply(lambda tokens: list(ngrams(tokens,1)))

In [17]:
#bigram
body["bigram"] = body["lemmas"].apply(lambda tokens: list(ngrams(tokens,2)))

In [18]:
body

,contributions_bodyText,tokens,lemmas,tags,propLen,nbPhrases,unigram,bigram
0,Mettre plutôt nos investissements dans les out...,"[Mettre, investissements, outils, cyberguerre,...","[mettre, investissement, outil, cyberguerre, n...","[(VERB, mettre), (NOUN, investissement), (NOUN...",9,1,"[(mettre,), (investissement,), (outil,), (cybe...","[(mettre, investissement), (investissement, ou..."
1,Mettre fin à une politique militaire d'ingéren...,"[Mettre, fin, politique, militaire, ingérence,...","[mettre, fin, politique, militaire, ingérence,...","[(VERB, mettre), (NOUN, fin), (NOUN, politique...",6,1,"[(mettre,), (fin,), (politique,), (militaire,)...","[(mettre, fin), (fin, politique), (politique, ..."
2,Création de groupes de citoyens non profession...,"[Création, groupes, citoyens, professionnels, ...","[création, groupe, citoyen, professionnel, cha...","[(NOUN, création), (NOUN, groupe), (NOUN, cito...",38,4,"[(création,), (groupe,), (citoyen,), (professi...","[(création, groupe), (groupe, citoyen), (citoy..."
3,Que les votes blancs soient pris en compte.Plu...,"[votes, blancs, soient, pris, compte, temps, p...","[vote, blancs, être, prendre, compte, temps, p...","[(NOUN, vote), (PROPN, blancs), (AUX, être), (...",23,3,"[(vote,), (blancs,), (être,), (prendre,), (com...","[(vote, blancs), (blancs, être), (être, prendr..."
4,Revaloriser les salaires de nos citoyens trava...,"[Revaloriser, salaires, citoyens, travaillant,...","[revaloriser, salaire, citoyen, travailler, pa...","[(VERB, revaloriser), (NOUN, salaire), (NOUN, ...",18,2,"[(revaloriser,), (salaire,), (citoyen,), (trav...","[(revaloriser, salaire), (salaire, citoyen), (..."
...,...,...,...,...,...,...,...,...
1395,Il serait utile qu'une police municipale exist...,"[utile, police, municipale, existe, communauté...","[utile, police, municipal, existe, communauté,...","[(ADJ, utile), (NOUN, police), (ADJ, municipal...",16,1,"[(utile,), (police,), (municipal,), (existe,),...","[(utile, police), (police, municipal), (munici..."
1396,Trafic de drogue : Saisie des biens du trafiqu...,"[Trafic, drogue, Saisie, biens, trafiquant, en...","[trafic, drogue, saisir, bien, trafiquer, ento...","[(NOUN, trafic), (NOUN, drogue), (VERB, saisir...",7,1,"[(trafic,), (drogue,), (saisir,), (bien,), (tr...","[(trafic, drogue), (drogue, saisir), (saisir, ..."
1397,"En ce qui concerne la justice, il est psycholo...","[concerne, justice, psychologiquement, nécessa...","[concerne, justice, psychologiquement, nécessa...","[(VERB, concerne), (NOUN, justice), (ADV, psyc...",60,6,"[(concerne,), (justice,), (psychologiquement,)...","[(concerne, justice), (justice, psychologiquem..."
1398,"Bonjour à tous, Citoyens, à l'heure actuelle, ...","[Bonjour, Citoyens, heure, actuelle, magistrat...","[bonjour, citoyen, heure, actuel, magistrat, c...","[(NOUN, bonjour), (NOUN, citoyen), (NOUN, heur...",90,3,"[(bonjour,), (citoyen,), (heure,), (actuel,), ...","[(bonjour, citoyen), (citoyen, heure), (heure,..."


In [19]:
unigram = flatten(body["unigram"].tolist())
bigrams = flatten(body["bigram"].tolist())

In [20]:
bigramCounter = Counter(bigrams)
unigramCounter = Counter(unigram)

In [21]:
unigramCounter.most_common(20)

[(('justice',), 513),
 (('pouvoir',), 313),
 (('faire',), 311),
 (('peine',), 292),
 (('police',), 256),
 (('loi',), 255),
 (('falloir',), 225),
 (('droit',), 224),
 (('prison',), 219),
 (('France',), 188),
 (('pays',), 187),
 (('citoyen',), 185),
 (('juge',), 185),
 (('devoir',), 184),
 (('cas',), 173),
 (('français',), 169),
 (('policier',), 156),
 (('ordre',), 153),
 (('public',), 150),
 (('politique',), 149)]

In [22]:
bigramCounter.most_common(20)

[(('force', 'ordre'), 69),
 (('gilet', 'jaune'), 40),
 (('peine', 'prison'), 38),
 (('service', 'militaire'), 29),
 (('peine', 'mort'), 27),
 (('maintien', 'ordre'), 27),
 (('mettre', 'place'), 22),
 (('droit', 'homme'), 21),
 (('service', 'public'), 19),
 (('police', 'municipal'), 18),
 (('casier', 'judiciaire'), 18),
 (('nationalité', 'français'), 17),
 (('mise', 'place'), 17),
 (('garde', 'sceau'), 16),
 (('supérieur', 'magistrature'), 15),
 (('cour', 'cassation'), 15),
 (('ministre', 'justice'), 15),
 (('procureur', 'république'), 15),
 (('intérêt', 'général'), 15),
 (('police', 'gendarmerie'), 14)]

In [23]:
def create_pos_dict(posList):
    pos_dict = dict()
    for tag, lemma in posList:
        pos_dict.setdefault(tag, []).append(lemma)
    return pos_dict

In [24]:
pos_data = create_pos_dict(flatten(body["tags"].tolist()))

In [25]:
pos_counters = dict()
for pos in pos_data:
    pos_counters[pos] = Counter(pos_data[pos])

In [26]:
def dsp_most_common_pos(dict_data, dict_counters, n):
    for tag in dict_data:
        print(f'Les {n} {spacy.explain(tag)} les plus fréquents :')
        for mot in dict_counters[tag].most_common(n):
            print(mot)
        print("="*40)

In [27]:
dsp_most_common_pos(pos_data, pos_counters, 15)

Les 15 verb les plus fréquents :
('faire', 311)
('falloir', 225)
('mettre', 144)
('prendre', 104)
('voir', 89)
('payer', 89)
('condamner', 88)
('permettre', 84)
('donner', 83)
('juger', 81)
('agir', 76)
('devoir', 74)
('supprimer', 70)
('passer', 66)
('créer', 61)
Les 15 noun les plus fréquents :
('justice', 510)
('peine', 288)
('police', 256)
('loi', 253)
('prison', 219)
('droit', 204)
('juge', 185)
('pays', 184)
('cas', 173)
('citoyen', 159)
('ordre', 152)
('service', 148)
('an', 143)
('policier', 129)
('personne', 128)
Les 15 adjective les plus fréquents :
('public', 143)
('français', 133)
('judiciaire', 111)
('militaire', 92)
('politique', 85)
('grand', 77)
('national', 76)
('social', 74)
('indépendant', 73)
('petit', 64)
('bon', 54)
('européen', 49)
('fiscal', 48)
('grave', 48)
('financier', 47)
Les 15 adverb les plus fréquents :
('déjà', 45)
('jamais', 42)
('mieux', 36)
('etc.', 29)
('autant', 28)
('actuellement', 28)
('également', 26)
('réellement', 20)
('pourtant', 19)
('vraime

In [28]:
# Création du dictionnaire
dictionary = corpora.Dictionary(body["lemmas"].tolist())

#Matrice Fréquence Terme-Document
TDmatrix = [dictionary.doc2bow(doc) for doc in body["lemmas"].tolist()]

In [29]:
start_time = time()
##
## Multicore Latent Dirichlet Allocation (LDA)
NUM_TOPICS = 3
lda_multicore_model = gensim.models.ldamulticore.LdaMulticore(TDmatrix, num_topics = NUM_TOPICS, id2word=dictionary,random_state=100,passes=10)

# Saving trained model
lda_multicore_model.save('LDA_police_multicore')

## Loading trained model
#lda_multicore_model = gensim.models.ldamodel.LdaModel.load('LDA_police_multicore')


## Print time taken to train the model
print("--- %s seconds ---" % (time() - start_time))

--- 12.984077215194702 seconds ---


In [30]:
# To plot at Jupyter notebook
pyLDAvis.enable_notebook()
plot = pyLDAvis.gensim.prepare(lda_multicore_model, TDmatrix, dictionary)
# Save pyLDA plot as html file
pyLDAvis.save_html(plot, 'LDA_police.html')
plot

/home/benjamin/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.005815 -0.065233       1        1  40.798523
0     -0.070153  0.028386       2        1  31.820553
1      0.064338  0.036848       3        1  27.380922, topic_info=    Category        Freq       Term       Total  loglift  logprob
280  Default  243.000000      peine  243.000000  30.0000  30.0000
531  Default  106.000000      force  106.000000  29.0000  29.0000
941  Default   84.000000      armée   84.000000  28.0000  28.0000
286  Default  182.000000     prison  182.000000  27.0000  27.0000
73   Default  154.000000       pays  154.000000  26.0000  26.0000
..       ...         ...        ...         ...      ...      ...
524   Topic3   31.938747    falloir  188.842194  -0.4818  -5.9535
286   Topic3   31.442560     prison  182.866608  -0.4653  -5.9692
20    Topic3   29.135683    citoyen  155.936798  -0.3822  -6.0454
110   Topic3   27.257841       juge  155.576492  -0.4465  -6.1120
13    Topic3   25.754459  politique  124.921738  -0.2838  -6.1688

[275 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
5769      1  0.840807  Castaner
2656      3  0.741749     Chine
82        2  0.863637   Conseil
82        3  0.115152   Conseil
1445      1  0.228645    Europe
...     ...       ...       ...
223       3  0.198134      état
1428      1  0.842042   évident
43        1  0.383007      être
43        2  0.258530      être
43        3  0.363857      être

[421 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

Avec $\lambda \approx 0.3$, il semblerait que:
- Classe 1 = Code de la route
- Classe 2 = Loi/Justice
- Classe 3 = Police/Armée

# Ignorer ce qu'il y a en dessous
$\downarrow \downarrow \downarrow \downarrow \downarrow$

In [ ]:
print(os.listdir("data/VD"))

In [ ]:
df = pd.read_csv(f"data/GDN/{sujets_GDN['fisc']}.csv")

In [ ]:
df

In [ ]:
ids = list(set(hd for hd in df.columns if "id" in hd))
ids

In [ ]:
test_id = "bf354656-2d2b-11e9-988a-fa163e5b3bfd"
ids.pop(0)

df.loc[df[ids[0]]==test_id]

In [ ]:
df[df.contributions_votesCount > 10]

In [ ]:
data2 = data2.drop("contributions_votes_author_id", axis=1)

In [ ]:
data2

In [ ]:
for key in sujets_GDN:
    frames = [pd.read_csv(f"data/GDN/{date}/{sujets_GDN[key]}.csv") for date in dates_GDN]
    exp = pd.concat(frames)
    exp.to_csv(f"data/GDN/{sujets_GDN[key]}.csv", index=False)
    
    

In [ ]:
for sujet in sujets_GDN:
    df = pd.read_csv(f"data/GDN/{sujets_GDN[sujet]}.csv")
    df.drop_duplicates().to_csv(f"data/GDN/{sujets_GDN[sujet]}.csv")

In [ ]:
import os

for file in os.listdir("data/VD"):
    full_path = os.path.join("data/VD", file)
    df = pd.read_csv(full_path)
    df.drop_duplicates().to_csv(full_path, index=False)

In [ ]:
s = set()

t = df[df.columns[12]].dropna()

for line in t:
    for val in line.split("|"):
        s.add(val)
    
s

In [ ]:
for col in df.columns[7:]:
    print(len(set(df[col].dropna())))
    print(set(df[col].dropna()))

In [ ]:
type_ex = [df.loc[df.type == t].iloc[0] for t in set(df["type"])]

new_df = pd.DataFrame(type_ex)

In [ ]:
new_df.to_csv("test.csv",index=False)